In [3]:
!pip install -U spacy -q

In [5]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.7                         
Location         /usr/local/lib/python3.12/dist-packages/spacy
Platform         Linux-6.1.123+-x86_64-with-glibc2.35
Python version   3.12.11                       
Pipelines        en_core_web_sm (3.8.0)        



In [18]:
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [19]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [6]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
nlp=spacy.blank("en")
db=DocBin()

In [10]:
import json
f=open('training_data.json')
TRAIN_DATA=json.load(f)

In [11]:
TRAIN_DATA

{'classes': ['ORG', 'PRODUCT', 'DATE', 'GPE', 'PERSON', 'MONEY'],
 'annotations': [['Apple Inc. announced the launch of its new iPhone 16 on September 12, 2025, in California. \r',
   {'entities': [[0, 10, 'ORG'],
     [43, 52, 'PRODUCT'],
     [56, 74, 'DATE'],
     [79, 90, 'GPE']]}],
  ['CEO Tim Cook stated that the company expects to generate over $10 billion in revenue by the end of the year.\r',
   {'entities': [[4, 12, 'PERSON'], [62, 73, 'MONEY']]}],
  ['Meanwhile, competitors like Samsung and Google are planning their own product releases in South Korea and the United States.\r',
   {'entities': [[28, 35, 'PRODUCT'],
     [40, 46, 'PRODUCT'],
     [90, 101, 'GPE'],
     [110, 123, 'GPE']]}],
  ['', {'entities': []}],
  ['', {'entities': []}]]}

In [13]:
for text,annot in tqdm(TRAIN_DATA['annotations']):
  doc=nlp.make_doc(text)
  ents=[]
  for start,end,label in annot["entities"]:
    span=doc.char_span(start,end,label=label,alignment_mode="contract")
    if span is None:
      print("Skipping entity")
    else:
        ents.append(span)
  doc.ents=ents
  db.add(doc)

db.to_disk("./training_data.spacy")


100%|██████████| 5/5 [00:00<00:00, 1113.67it/s]


In [14]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [22]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy


ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     11.73    0.00    0.00    0.00    0.00
 66     200         16.10    395.20  100.00  100.00  100.00    1.00
133     400          0.00      0.00  100.00  100.00  100.00    1.00
200     600          0.00      0.00  100.00  100.00  100.00    1.00
266     800          0.00      0.00  100.00  100.00  100.00    1.00
333    1000          0.00      0.00  100.00  100.00  100.00    1.00
400    1200          0.00      0.00  100.00  100.00  100.00    1.00
466    1400          0.00      0.00  100.00  100.00  100.00    1.00
533    1600          0.00      0.00 

In [23]:
nlp_ner =spacy.load("/content/model-best")

In [27]:
doc =nlp_ner('''Amazon introduced its revolutionary Kindle X3 on November 20, 2025, during a special event in Seattle.
Founder Jeff Bezos highlighted that the company aims to surpass $15 billion in sales by mid-2026.
Meanwhile, competitors such as Sony and Lenovo are preparing their own e-reader launches in Canada and France.
''')

In [28]:
spacy.displacy.render(doc,style="ent",jupyter=True)